<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/CA-labelingCleanVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler as ss
import itertools
from scipy import linalg
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
import numpy as np
import matplotlib.pyplot as plt

# Load Combined Dataset

In [59]:
CA_combined_data = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/CA/CA_combined_Sep25.csv',low_memory=False)
CA_combined_data.head()

,Date,fips_x,County Name,population,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,m50,m50_index,population_density,mask_rule_active,mask_wearing_percent,New cases
0,2020-03-01,6001,Alameda County,1671329,6.0,9.0,15.0,1.0,4.0,0.0,-7.65,3.684,58,2261.547725,0,0.0,0
1,2020-03-02,6001,Alameda County,1671329,4.0,7.0,7.0,-1.0,3.0,0.0,6.76,6.182,98,2261.547725,0,0.0,0
2,2020-03-03,6001,Alameda County,1671329,5.0,12.0,41.0,0.0,1.0,-1.0,13.19,6.271,100,2261.547725,0,0.0,1
3,2020-03-04,6001,Alameda County,1671329,1.0,6.0,23.0,-3.0,1.0,0.0,10.67,6.209,99,2261.547725,0,0.0,0
4,2020-03-05,6001,Alameda County,1671329,1.0,6.0,12.0,-5.0,-1.0,1.0,11.64,6.654,106,2261.547725,0,0.0,0


In [60]:
CA_combined_data['New Cases/1k population'] = CA_combined_data['New cases']*1000/ CA_combined_data['population']
CA_combined_data.tail(2)

,Date,fips_x,County Name,population,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,m50,m50_index,population_density,mask_rule_active,mask_wearing_percent,New cases,New Cases/1k population
9194,2020-08-20,6115,Yuba County,78668,0.0,0.0,0.0,0.0,-33.0,8.0,59.48,4.096,43,124.506204,1,0.669,0,0.000000
9195,2020-08-21,6115,Yuba County,78668,0.0,0.0,0.0,0.0,-33.0,10.0,84.31,5.820,61,124.506204,1,0.669,8,0.101693


## 1.Filter-Los Angeles County

In [61]:
CA_combined_data_test_la = CA_combined_data[CA_combined_data['County Name']== 'Los Angeles County']
CA_combined_data_test_la.head(3)

,Date,fips_x,County Name,population,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,m50,m50_index,population_density,mask_rule_active,mask_wearing_percent,New cases,New Cases/1k population
2924,2020-03-01,6037,Los Angeles County,10039107,4.0,3.0,-5.0,-3.0,1.0,0.0,-4.36,3.996,60,2473.978284,0,0.0,0,0.0
2925,2020-03-02,6037,Los Angeles County,10039107,5.0,5.0,8.0,1.0,4.0,-1.0,7.51,6.502,98,2473.978284,0,0.0,0,0.0
2926,2020-03-03,6037,Los Angeles County,10039107,8.0,7.0,32.0,2.0,3.0,-1.0,17.24,6.591,99,2473.978284,0,0.0,0,0.0


In [62]:
new_cases_1k_la_list = CA_combined_data_test_la ['New Cases/1k population'].values

### Finding mean, std. dev

In [63]:
meanVal= np.mean(new_cases_1k_la_list, axis=0)
stdVal = np.std(new_cases_1k_la_list, axis=0)
print('Mean: ', np.mean(new_cases_1k_la_list, axis=0))
print('Standard Deviation:', np.std(new_cases_1k_la_list, axis=0))

Mean:  0.13074045747898344
Standard Deviation: 0.11493892995460023


### Method to compute normal distribution to label the data 

In [64]:
  def determine_growth_label_stdev(i,meanVal,stdVal):
    if (i > (meanVal + (stdVal * 1) / 3)):
        #print("Label: ", i, "drastic_growth")
        return 'drastic_growth'
    elif ( i > (meanVal) and i <= (meanVal + (stdVal * 1) / 3)):
        #print("Label: ", i, "mild_growth")
        return 'mild_growth'
    elif (i >= (meanVal - (stdVal * 1) / 3) and i <= meanVal):
        #print("Label: ", i, "flat_growth")
        return 'flat_growth'
    elif (i < (meanVal - (stdVal * 1) / 3) and i >= (meanVal - (stdVal * 2) / 3)):
        #print("Label: ", i, "mild_decrement")
        return 'mild_decrement'
    elif (i < (meanVal - (stdVal * 2) / 3)):
        #print("Label: ", i, "drastic_decrement")
        return 'drastic_decrement'
    

In [65]:
CA_combined_data_test_la['growth_label'] = CA_combined_data_test_la['New Cases/1k population'].apply(lambda x: determine_growth_label_stdev(x,meanVal,stdVal) )
CA_combined_data_test_la.head(4)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,fips_x,County Name,population,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,m50,m50_index,population_density,mask_rule_active,mask_wearing_percent,New cases,New Cases/1k population,growth_label
2924,2020-03-01,6037,Los Angeles County,10039107,4.0,3.0,-5.0,-3.0,1.0,0.0,-4.36,3.996,60,2473.978284,0,0.0,0,0.000000,drastic_decrement
2925,2020-03-02,6037,Los Angeles County,10039107,5.0,5.0,8.0,1.0,4.0,-1.0,7.51,6.502,98,2473.978284,0,0.0,0,0.000000,drastic_decrement
2926,2020-03-03,6037,Los Angeles County,10039107,8.0,7.0,32.0,2.0,3.0,-1.0,17.24,6.591,99,2473.978284,0,0.0,0,0.000000,drastic_decrement
2927,2020-03-04,6037,Los Angeles County,10039107,4.0,6.0,9.0,-1.0,2.0,0.0,13.82,6.595,100,2473.978284,0,0.0,6,0.000598,drastic_decrement


In [66]:
print(CA_combined_data_test_la[CA_combined_data_test_la['growth_label'] == 'drastic_growth'].shape)
print(CA_combined_data_test_la[CA_combined_data_test_la['growth_label'] == 'mild_decrement'].shape)
print(CA_combined_data_test_la[CA_combined_data_test_la['growth_label'] == 'drastic_decrement'].shape)
print(CA_combined_data_test_la[CA_combined_data_test_la['growth_label'] == 'mild_growth'].shape)
print(CA_combined_data_test_la[CA_combined_data_test_la['growth_label'] == 'flat_growth'].shape)


(55, 19)
(26, 19)
(50, 19)
(17, 19)
(24, 19)


In [67]:
print(CA_combined_data_test_la[CA_combined_data_test_la['growth_label'] == 'drastic_growth'].shape)
print(CA_combined_data_test_la[CA_combined_data_test_la['growth_label'] == 'mild_decrement'].shape)
print(CA_combined_data_test_la[CA_combined_data_test_la['growth_label'] == 'drastic_decrement'].shape)
print(CA_combined_data_test_la[CA_combined_data_test_la['growth_label'] == 'mild_growth'].shape)
print(CA_combined_data_test_la[CA_combined_data_test_la['growth_label'] == 'flat_growth'].shape)


(55, 19)
(26, 19)
(50, 19)
(17, 19)
(24, 19)


### Export CSV File

In [68]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/CA/'
filename = 'CA_labeledData_LosAngeles_'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
CA_combined_data_test_la.to_csv(location+filename,index=False)

CA_labeledData_LosAngeles_Oct11.csv


## 2.Filter-San Diego County

In [69]:
CA_combined_data_test_sd = CA_combined_data[CA_combined_data['County Name']== 'San Diego County']
CA_combined_data_test_sd.head(3)

,Date,fips_x,County Name,population,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,m50,m50_index,population_density,mask_rule_active,mask_wearing_percent,New cases,New Cases/1k population
5756,2020-03-01,6073,San Diego County,3338330,2.0,-1.0,-20.0,-4.0,2.0,0.0,-4.99,4.555,59,793.587741,0,0.0,0,0.0
5757,2020-03-02,6073,San Diego County,3338330,3.0,1.0,-3.0,-3.0,6.0,-1.0,10.28,7.409,96,793.587741,0,0.0,0,0.0
5758,2020-03-03,6073,San Diego County,3338330,6.0,5.0,19.0,2.0,4.0,-1.0,19.54,7.627,99,793.587741,0,0.0,0,0.0


In [70]:
new_cases_1k_la_list = CA_combined_data_test_sd ['New Cases/1k population'].values

### Finding mean, std. dev

In [71]:
meanVal= np.mean(new_cases_1k_la_list, axis=0)
stdVal = np.std(new_cases_1k_la_list, axis=0)
print('Mean: ', np.mean(new_cases_1k_la_list, axis=0))
print('Standard Deviation:', np.std(new_cases_1k_la_list, axis=0))

Mean:  0.062444186861568926
Standard Deviation: 0.0715759305704848


### Method to compute normal distribution to label the data 

In [72]:
  def determine_growth_label_stdev(i,meanVal,stdVal):
    if (i > (meanVal + (stdVal * 1) / 3)):
        #print("Label: ", i, "drastic_growth")
        return 'drastic_growth'
    elif ( i > (meanVal) and i <= (meanVal + (stdVal * 1) / 3)):
        #print("Label: ", i, "mild_growth")
        return 'mild_growth'
    elif (i >= (meanVal - (stdVal * 1) / 3) and i <= meanVal):
        #print("Label: ", i, "flat_growth")
        return 'flat_growth'
    elif (i < (meanVal - (stdVal * 1) / 3) and i >= (meanVal - (stdVal * 2) / 3)):
        #print("Label: ", i, "mild_decrement")
        return 'mild_decrement'
    elif (i < (meanVal - (stdVal * 2) / 3)):
        #print("Label: ", i, "drastic_decrement")
        return 'drastic_decrement'
    

In [73]:
CA_combined_data_test_sd['growth_label'] = CA_combined_data_test_sd['New Cases/1k population'].apply(lambda x: determine_growth_label_stdev(x,meanVal,stdVal) )
CA_combined_data_test_sd.head(4)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,fips_x,County Name,population,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,m50,m50_index,population_density,mask_rule_active,mask_wearing_percent,New cases,New Cases/1k population,growth_label
5756,2020-03-01,6073,San Diego County,3338330,2.0,-1.0,-20.0,-4.0,2.0,0.0,-4.99,4.555,59,793.587741,0,0.0,0,0.0,drastic_decrement
5757,2020-03-02,6073,San Diego County,3338330,3.0,1.0,-3.0,-3.0,6.0,-1.0,10.28,7.409,96,793.587741,0,0.0,0,0.0,drastic_decrement
5758,2020-03-03,6073,San Diego County,3338330,6.0,5.0,19.0,2.0,4.0,-1.0,19.54,7.627,99,793.587741,0,0.0,0,0.0,drastic_decrement
5759,2020-03-04,6073,San Diego County,3338330,3.0,2.0,14.0,0.0,3.0,0.0,15.61,7.675,100,793.587741,0,0.0,0,0.0,drastic_decrement


In [74]:
print(CA_combined_data_test_sd[CA_combined_data_test_sd['growth_label'] == 'drastic_growth'].shape)
print(CA_combined_data_test_sd[CA_combined_data_test_sd['growth_label'] == 'mild_decrement'].shape)
print(CA_combined_data_test_sd[CA_combined_data_test_sd['growth_label'] == 'drastic_decrement'].shape)
print(CA_combined_data_test_sd[CA_combined_data_test_sd['growth_label'] == 'mild_growth'].shape)
print(CA_combined_data_test_sd[CA_combined_data_test_sd['growth_label'] == 'flat_growth'].shape)


(49, 19)
(36, 19)
(55, 19)
(15, 19)
(17, 19)


In [75]:
print(CA_combined_data_test_sd[CA_combined_data_test_sd['growth_label'] == 'drastic_growth'].shape)
print(CA_combined_data_test_sd[CA_combined_data_test_sd['growth_label'] == 'mild_decrement'].shape)
print(CA_combined_data_test_sd[CA_combined_data_test_sd['growth_label'] == 'drastic_decrement'].shape)
print(CA_combined_data_test_sd[CA_combined_data_test_sd['growth_label'] == 'mild_growth'].shape)
print(CA_combined_data_test_sd[CA_combined_data_test_sd['growth_label'] == 'flat_growth'].shape)


(49, 19)
(36, 19)
(55, 19)
(15, 19)
(17, 19)


### Export CSV File

In [76]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/CA/'
filename = 'CA_labeledData_SanDiego_'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
CA_combined_data_test_sd.to_csv(location+filename,index=False)

CA_labeledData_SanDiego_Oct11.csv


## 3.Filter-San Bernardino County

In [77]:
CA_combined_data_test_sb = CA_combined_data[CA_combined_data['County Name']== 'San Bernardino County']
CA_combined_data_test_sb.head(3)

,Date,fips_x,County Name,population,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,m50,m50_index,population_density,mask_rule_active,mask_wearing_percent,New cases,New Cases/1k population
5584,2020-03-01,6071,San Bernardino County,2180085,5.0,-2.0,-22.0,2.0,-1.0,0.0,10.34,4.508,57,108.694796,0,0.0,0,0.0
5585,2020-03-02,6071,San Bernardino County,2180085,7.0,3.0,3.0,-4.0,3.0,0.0,9.71,7.678,98,108.694796,0,0.0,0,0.0
5586,2020-03-03,6071,San Bernardino County,2180085,11.0,10.0,26.0,-1.0,2.0,-1.0,16.47,7.657,98,108.694796,0,0.0,0,0.0


In [78]:
new_cases_1k_la_list = CA_combined_data_test_sb ['New Cases/1k population'].values

### Finding mean, std. dev

In [79]:
meanVal= np.mean(new_cases_1k_la_list, axis=0)
stdVal = np.std(new_cases_1k_la_list, axis=0)
print('Mean: ', np.mean(new_cases_1k_la_list, axis=0))
print('Standard Deviation:', np.std(new_cases_1k_la_list, axis=0))

Mean:  0.11654922138463664
Standard Deviation: 0.1816622121691984


### Method to compute normal distribution to label the data 

In [80]:
  def determine_growth_label_stdev(i,meanVal,stdVal):
    if (i > (meanVal + (stdVal * 1) / 3)):
        #print("Label: ", i, "drastic_growth")
        return 'drastic_growth'
    elif ( i > (meanVal) and i <= (meanVal + (stdVal * 1) / 3)):
        #print("Label: ", i, "mild_growth")
        return 'mild_growth'
    elif (i >= (meanVal - (stdVal * 1) / 3) and i <= meanVal):
        #print("Label: ", i, "flat_growth")
        return 'flat_growth'
    elif (i < (meanVal - (stdVal * 1) / 3) and i >= (meanVal - (stdVal * 2) / 3)):
        #print("Label: ", i, "mild_decrement")
        return 'mild_decrement'
    elif (i < (meanVal - (stdVal * 2) / 3)):
        #print("Label: ", i, "drastic_decrement")
        return 'drastic_decrement'
    

In [81]:
CA_combined_data_test_sb['growth_label'] = CA_combined_data_test_sb['New Cases/1k population'].apply(lambda x: determine_growth_label_stdev(x,meanVal,stdVal) )
CA_combined_data_test_sb.head(4)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,fips_x,County Name,population,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,m50,m50_index,population_density,mask_rule_active,mask_wearing_percent,New cases,New Cases/1k population,growth_label
5584,2020-03-01,6071,San Bernardino County,2180085,5.0,-2.0,-22.0,2.0,-1.0,0.0,10.34,4.508,57,108.694796,0,0.0,0,0.0,mild_decrement
5585,2020-03-02,6071,San Bernardino County,2180085,7.0,3.0,3.0,-4.0,3.0,0.0,9.71,7.678,98,108.694796,0,0.0,0,0.0,mild_decrement
5586,2020-03-03,6071,San Bernardino County,2180085,11.0,10.0,26.0,-1.0,2.0,-1.0,16.47,7.657,98,108.694796,0,0.0,0,0.0,mild_decrement
5587,2020-03-04,6071,San Bernardino County,2180085,8.0,7.0,20.0,0.0,2.0,-1.0,18.70,7.802,100,108.694796,0,0.0,0,0.0,mild_decrement


In [82]:
print(CA_combined_data_test_sb[CA_combined_data_test_sb['growth_label'] == 'drastic_growth'].shape)
print(CA_combined_data_test_sb[CA_combined_data_test_sb['growth_label'] == 'mild_decrement'].shape)
print(CA_combined_data_test_sb[CA_combined_data_test_sb['growth_label'] == 'drastic_decrement'].shape)
print(CA_combined_data_test_sb[CA_combined_data_test_sb['growth_label'] == 'mild_growth'].shape)
print(CA_combined_data_test_sb[CA_combined_data_test_sb['growth_label'] == 'flat_growth'].shape)


(40, 19)
(88, 19)
(1, 19)
(13, 19)
(30, 19)


In [83]:
print(CA_combined_data_test_sb[CA_combined_data_test_sb['growth_label'] == 'drastic_growth'].shape)
print(CA_combined_data_test_sb[CA_combined_data_test_sb['growth_label'] == 'mild_decrement'].shape)
print(CA_combined_data_test_sb[CA_combined_data_test_sb['growth_label'] == 'drastic_decrement'].shape)
print(CA_combined_data_test_sb[CA_combined_data_test_sb['growth_label'] == 'mild_growth'].shape)
print(CA_combined_data_test_sb[CA_combined_data_test_sb['growth_label'] == 'flat_growth'].shape)


(40, 19)
(88, 19)
(1, 19)
(13, 19)
(30, 19)


### Export CSV File

In [84]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/CA/'
filename = 'CA_labeledData_SanBernardino_'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
CA_combined_data_test_sb.to_csv(location+filename,index=False)

CA_labeledData_SanBernardino_Oct11.csv


# Loading Inertia

In [85]:
CA_combined_sd_inertia_data = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/CA/CA_socialDistancingInertiaData_Sep25.csv',low_memory=False)
CA_combined_sd_inertia_data.head()

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,CTFIPS,CTNAME,STFIPS,Social distancing index,% staying home,Trips/person,% out-of-county trips,% out-of-state trips,Miles/person,Work trips/person,Non-work trips/person,New COVID cases,Population,% change in consumption,date,Transit mode share,% people older than 60,Median income,% African Americans,% Hispanic Americans,% Male,Population density,Employment density,# hot spots/1000 people,Hospital beds/1000 people,ICUs/1000 people,# contact tracing workers/1000 people,# contact tracing workers/1000 people rank,% people older than 60 rank,COVID exposure/1000 people,#days: decreasing ILI cases,Unemployment claims/1000 people,Unemployment rate,% working from home,Cumulative inflation rate,COVID death rate,New cases/1000 people,Active cases/1000 people,#days: decreasing COVID cases,% hospital bed utilization,Testing capacity,Tests done/1000 people,% ICU utilization,Ventilator shortage,Imported COVID cases
0,2020-01-22,California,6001,0,0,0,0,0,6001,Alameda County,6,18,20,3.29,29.4,0.3,35.9,0.55,2.74,0,1666753,-2.9,2020-01-22,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0
1,2020-01-23,California,6001,0,0,0,0,0,6001,Alameda County,6,15,19,3.36,29.6,0.3,35.4,0.56,2.80,0,1666753,-0.8,2020-01-23,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0
2,2020-01-24,California,6001,0,0,0,0,0,6001,Alameda County,6,15,19,3.48,29.0,0.3,39.1,0.53,2.94,0,1666753,4.1,2020-01-24,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0
3,2020-01-25,California,6001,0,0,0,0,0,6001,Alameda County,6,30,24,3.16,26.6,0.3,36.3,0.27,2.88,0,1666753,2.0,2020-01-25,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,1,54.0,0.0,0.0,0.0,0,0
4,2020-01-26,California,6001,0,0,0,0,0,6001,Alameda County,6,41,28,2.80,25.6,0.4,35.4,0.24,2.56,0,1666753,-9.3,2020-01-26,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,0.0,0,1.1,3.9,5.7,0.3,0.0,0.0,0.0,2,54.0,0.0,0.0,0.0,0,0


## 1.Filter-Los Angeles County

In [86]:
CA_combined_sd_inertia_data_la = CA_combined_sd_inertia_data[CA_combined_sd_inertia_data['CTNAME']== 'Los Angeles County']
CA_combined_sd_inertia_data_la.head(3)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,CTFIPS,CTNAME,STFIPS,Social distancing index,% staying home,Trips/person,% out-of-county trips,% out-of-state trips,Miles/person,Work trips/person,Non-work trips/person,New COVID cases,Population,% change in consumption,date,Transit mode share,% people older than 60,Median income,% African Americans,% Hispanic Americans,% Male,Population density,Employment density,# hot spots/1000 people,Hospital beds/1000 people,ICUs/1000 people,# contact tracing workers/1000 people,# contact tracing workers/1000 people rank,% people older than 60 rank,COVID exposure/1000 people,#days: decreasing ILI cases,Unemployment claims/1000 people,Unemployment rate,% working from home,Cumulative inflation rate,COVID death rate,New cases/1000 people,Active cases/1000 people,#days: decreasing COVID cases,% hospital bed utilization,Testing capacity,Tests done/1000 people,% ICU utilization,Ventilator shortage,Imported COVID cases
3816,2020-01-22,California,6037,0,0,0,0,0,6037,Los Angeles County,6,18,19,3.34,10.9,0.3,32.5,0.62,2.72,0,10105518,-1.7,2020-01-22,6.04,18,64251,7.9,48.5,49.28,2125,972,140,2.05,0.22,NaN,NaN,9,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0
3817,2020-01-23,California,6037,0,0,0,0,0,6037,Los Angeles County,6,15,19,3.43,10.9,0.4,34.7,0.63,2.80,0,10105518,1.2,2020-01-23,6.04,18,64251,7.9,48.5,49.28,2125,972,140,2.05,0.22,NaN,NaN,9,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0
3818,2020-01-24,California,6037,0,0,0,0,0,6037,Los Angeles County,6,14,18,3.53,11.1,0.4,37.4,0.60,2.93,0,10105518,5.9,2020-01-24,6.04,18,64251,7.9,48.5,49.28,2125,972,140,2.05,0.22,NaN,NaN,9,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0


In [87]:
new_cases_1k_la_list = CA_combined_sd_inertia_data_la['New cases/1000 people'].values

### Finding mean, std. dev

In [88]:
meanVal= np.mean(new_cases_1k_la_list, axis=0)
stdVal = np.std(new_cases_1k_la_list, axis=0)
print('Mean: ', np.mean(new_cases_1k_la_list, axis=0))
print('Standard Deviation:', np.std(new_cases_1k_la_list, axis=0))

Mean:  0.10537877358490563
Standard Deviation: 0.10129884785989024


### Method to compute normal distribution to label the data 

In [89]:
  def determine_growth_label_stdev(i,meanVal,stdVal):
    if (i > (meanVal + (stdVal * 1) / 3)):
        #print("Label: ", i, "drastic_growth")
        return 'drastic_growth'
    elif ( i > (meanVal) and i <= (meanVal + (stdVal * 1) / 3)):
        #print("Label: ", i, "mild_growth")
        return 'mild_growth'
    elif (i >= (meanVal - (stdVal * 1) / 3) and i <= meanVal):
        #print("Label: ", i, "flat_growth")
        return 'flat_growth'
    elif (i < (meanVal - (stdVal * 1) / 3) and i >= (meanVal - (stdVal * 2) / 3)):
        #print("Label: ", i, "mild_decrement")
        return 'mild_decrement'
    elif (i < (meanVal - (stdVal * 2) / 3)):
        #print("Label: ", i, "drastic_decrement")
        return 'drastic_decrement'
    

In [90]:
CA_combined_sd_inertia_data_la['growth_label'] = CA_combined_sd_inertia_data_la['New cases/1000 people'].apply(lambda x: determine_growth_label_stdev(x,meanVal,stdVal) )
CA_combined_sd_inertia_data_la.head(4)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,CTFIPS,CTNAME,STFIPS,Social distancing index,% staying home,Trips/person,% out-of-county trips,% out-of-state trips,Miles/person,Work trips/person,Non-work trips/person,New COVID cases,Population,% change in consumption,date,Transit mode share,% people older than 60,Median income,% African Americans,% Hispanic Americans,% Male,Population density,Employment density,# hot spots/1000 people,Hospital beds/1000 people,ICUs/1000 people,# contact tracing workers/1000 people,# contact tracing workers/1000 people rank,% people older than 60 rank,COVID exposure/1000 people,#days: decreasing ILI cases,Unemployment claims/1000 people,Unemployment rate,% working from home,Cumulative inflation rate,COVID death rate,New cases/1000 people,Active cases/1000 people,#days: decreasing COVID cases,% hospital bed utilization,Testing capacity,Tests done/1000 people,% ICU utilization,Ventilator shortage,Imported COVID cases,growth_label
3816,2020-01-22,California,6037,0,0,0,0,0,6037,Los Angeles County,6,18,19,3.34,10.9,0.3,32.5,0.62,2.72,0,10105518,-1.7,2020-01-22,6.04,18,64251,7.9,48.5,49.28,2125,972,140,2.05,0.22,NaN,NaN,9,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,drastic_decrement
3817,2020-01-23,California,6037,0,0,0,0,0,6037,Los Angeles County,6,15,19,3.43,10.9,0.4,34.7,0.63,2.80,0,10105518,1.2,2020-01-23,6.04,18,64251,7.9,48.5,49.28,2125,972,140,2.05,0.22,NaN,NaN,9,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,drastic_decrement
3818,2020-01-24,California,6037,0,0,0,0,0,6037,Los Angeles County,6,14,18,3.53,11.1,0.4,37.4,0.60,2.93,0,10105518,5.9,2020-01-24,6.04,18,64251,7.9,48.5,49.28,2125,972,140,2.05,0.22,NaN,NaN,9,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,drastic_decrement
3819,2020-01-25,California,6037,0,0,0,0,0,6037,Los Angeles County,6,25,22,3.24,11.8,0.4,34.6,0.31,2.93,0,10105518,5.9,2020-01-25,6.04,18,64251,7.9,48.5,49.28,2125,972,140,2.05,0.22,NaN,NaN,9,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,1,54.0,0.0,0.0,0.0,0,0,drastic_decrement


### Export CSV File

In [91]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/CA/'
filename = 'CA_sd_inertia_labeledData_LosAngeles_'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
CA_combined_sd_inertia_data_la.to_csv(location+filename,index=False)

CA_sd_inertia_labeledData_LosAngeles_Oct11.csv


In [92]:
print(CA_combined_sd_inertia_data_la[CA_combined_sd_inertia_data_la['growth_label'] == 'drastic_growth'].shape)
print(CA_combined_sd_inertia_data_la[CA_combined_sd_inertia_data_la['growth_label'] == 'mild_decrement'].shape)
print(CA_combined_sd_inertia_data_la[CA_combined_sd_inertia_data_la['growth_label'] == 'drastic_decrement'].shape)
print(CA_combined_sd_inertia_data_la[CA_combined_sd_inertia_data_la['growth_label'] == 'mild_growth'].shape)
print(CA_combined_sd_inertia_data_la[CA_combined_sd_inertia_data_la['growth_label'] == 'flat_growth'].shape)


(64, 54)
(23, 54)
(73, 54)
(29, 54)
(23, 54)


## 2.Filter-San Diego County

In [93]:
CA_combined_sd_inertia_data_sd = CA_combined_sd_inertia_data[CA_combined_sd_inertia_data['CTNAME']== 'San Diego County']
CA_combined_sd_inertia_data_sd.head(3)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,CTFIPS,CTNAME,STFIPS,Social distancing index,% staying home,Trips/person,% out-of-county trips,% out-of-state trips,Miles/person,Work trips/person,Non-work trips/person,New COVID cases,Population,% change in consumption,date,Transit mode share,% people older than 60,Median income,% African Americans,% Hispanic Americans,% Male,Population density,Employment density,# hot spots/1000 people,Hospital beds/1000 people,ICUs/1000 people,# contact tracing workers/1000 people,# contact tracing workers/1000 people rank,% people older than 60 rank,COVID exposure/1000 people,#days: decreasing ILI cases,Unemployment claims/1000 people,Unemployment rate,% working from home,Cumulative inflation rate,COVID death rate,New cases/1000 people,Active cases/1000 people,#days: decreasing COVID cases,% hospital bed utilization,Testing capacity,Tests done/1000 people,% ICU utilization,Ventilator shortage,Imported COVID cases
7632,2020-01-22,California,6073,0,0,0,0,0,6073,San Diego County,6,17,18,3.45,4.6,0.7,34.9,0.60,2.84,0,3343364,0.8,2020-01-22,2.9,19,74855,4.7,33.5,50.32,730,407,143,2.05,0.22,NaN,NaN,12,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0
7633,2020-01-23,California,6073,0,0,0,0,0,6073,San Diego County,6,13,17,3.50,5.1,0.7,38.2,0.61,2.88,0,3343364,2.2,2020-01-23,2.9,19,74855,4.7,33.5,50.32,730,407,143,2.05,0.22,NaN,NaN,12,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0
7634,2020-01-24,California,6073,0,0,0,0,0,6073,San Diego County,6,13,17,3.63,5.4,0.7,39.7,0.58,3.04,0,3343364,7.9,2020-01-24,2.9,19,74855,4.7,33.5,50.32,730,407,143,2.05,0.22,NaN,NaN,12,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0


In [94]:
new_cases_1k_la_list = CA_combined_sd_inertia_data_sd['New cases/1000 people'].values

### Finding mean, std. dev

In [95]:
meanVal= np.mean(new_cases_1k_la_list, axis=0)
stdVal = np.std(new_cases_1k_la_list, axis=0)
print('Mean: ', np.mean(new_cases_1k_la_list, axis=0))
print('Standard Deviation:', np.std(new_cases_1k_la_list, axis=0))

Mean:  0.04982216981132076
Standard Deviation: 0.05325510896750634


### Method to compute normal distribution to label the data 

In [96]:
  def determine_growth_label_stdev(i,meanVal,stdVal):
    if (i > (meanVal + (stdVal * 1) / 3)):
        #print("Label: ", i, "drastic_growth")
        return 'drastic_growth'
    elif ( i > (meanVal) and i <= (meanVal + (stdVal * 1) / 3)):
        #print("Label: ", i, "mild_growth")
        return 'mild_growth'
    elif (i >= (meanVal - (stdVal * 1) / 3) and i <= meanVal):
        #print("Label: ", i, "flat_growth")
        return 'flat_growth'
    elif (i < (meanVal - (stdVal * 1) / 3) and i >= (meanVal - (stdVal * 2) / 3)):
        #print("Label: ", i, "mild_decrement")
        return 'mild_decrement'
    elif (i < (meanVal - (stdVal * 2) / 3)):
        #print("Label: ", i, "drastic_decrement")
        return 'drastic_decrement'
    

In [97]:
CA_combined_sd_inertia_data_sd['growth_label'] = CA_combined_sd_inertia_data_sd['New cases/1000 people'].apply(lambda x: determine_growth_label_stdev(x,meanVal,stdVal) )
CA_combined_sd_inertia_data_sd.head(4)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,CTFIPS,CTNAME,STFIPS,Social distancing index,% staying home,Trips/person,% out-of-county trips,% out-of-state trips,Miles/person,Work trips/person,Non-work trips/person,New COVID cases,Population,% change in consumption,date,Transit mode share,% people older than 60,Median income,% African Americans,% Hispanic Americans,% Male,Population density,Employment density,# hot spots/1000 people,Hospital beds/1000 people,ICUs/1000 people,# contact tracing workers/1000 people,# contact tracing workers/1000 people rank,% people older than 60 rank,COVID exposure/1000 people,#days: decreasing ILI cases,Unemployment claims/1000 people,Unemployment rate,% working from home,Cumulative inflation rate,COVID death rate,New cases/1000 people,Active cases/1000 people,#days: decreasing COVID cases,% hospital bed utilization,Testing capacity,Tests done/1000 people,% ICU utilization,Ventilator shortage,Imported COVID cases,growth_label
7632,2020-01-22,California,6073,0,0,0,0,0,6073,San Diego County,6,17,18,3.45,4.6,0.7,34.9,0.60,2.84,0,3343364,0.8,2020-01-22,2.9,19,74855,4.7,33.5,50.32,730,407,143,2.05,0.22,NaN,NaN,12,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,drastic_decrement
7633,2020-01-23,California,6073,0,0,0,0,0,6073,San Diego County,6,13,17,3.50,5.1,0.7,38.2,0.61,2.88,0,3343364,2.2,2020-01-23,2.9,19,74855,4.7,33.5,50.32,730,407,143,2.05,0.22,NaN,NaN,12,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,drastic_decrement
7634,2020-01-24,California,6073,0,0,0,0,0,6073,San Diego County,6,13,17,3.63,5.4,0.7,39.7,0.58,3.04,0,3343364,7.9,2020-01-24,2.9,19,74855,4.7,33.5,50.32,730,407,143,2.05,0.22,NaN,NaN,12,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,drastic_decrement
7635,2020-01-25,California,6073,0,0,0,0,0,6073,San Diego County,6,25,22,3.26,6.3,0.8,37.0,0.30,2.96,0,3343364,5.0,2020-01-25,2.9,19,74855,4.7,33.5,50.32,730,407,143,2.05,0.22,NaN,NaN,12,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,1,54.0,0.0,0.0,0.0,0,0,drastic_decrement


### Export CSV File

In [98]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/CA/'
filename = 'CA_sd_inertia_labeledData_sandiego_'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
CA_combined_sd_inertia_data_sd.to_csv(location+filename,index=False)

CA_sd_inertia_labeledData_sandiego_Oct11.csv


In [99]:
print(CA_combined_sd_inertia_data_sd[CA_combined_sd_inertia_data_sd['growth_label'] == 'drastic_growth'].shape)
print(CA_combined_sd_inertia_data_sd[CA_combined_sd_inertia_data_sd['growth_label'] == 'mild_decrement'].shape)
print(CA_combined_sd_inertia_data_sd[CA_combined_sd_inertia_data_sd['growth_label'] == 'drastic_decrement'].shape)
print(CA_combined_sd_inertia_data_sd[CA_combined_sd_inertia_data_sd['growth_label'] == 'mild_growth'].shape)
print(CA_combined_sd_inertia_data_sd[CA_combined_sd_inertia_data_sd['growth_label'] == 'flat_growth'].shape)


(59, 54)
(37, 54)
(66, 54)
(11, 54)
(39, 54)


## 3.Filter-San Bernardino County

In [100]:
CA_combined_sd_inertia_data_sb = CA_combined_sd_inertia_data[CA_combined_sd_inertia_data['CTNAME']== 'San Bernardino County']
CA_combined_sd_inertia_data_sb.head(3)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,CTFIPS,CTNAME,STFIPS,Social distancing index,% staying home,Trips/person,% out-of-county trips,% out-of-state trips,Miles/person,Work trips/person,Non-work trips/person,New COVID cases,Population,% change in consumption,date,Transit mode share,% people older than 60,Median income,% African Americans,% Hispanic Americans,% Male,Population density,Employment density,# hot spots/1000 people,Hospital beds/1000 people,ICUs/1000 people,# contact tracing workers/1000 people,# contact tracing workers/1000 people rank,% people older than 60 rank,COVID exposure/1000 people,#days: decreasing ILI cases,Unemployment claims/1000 people,Unemployment rate,% working from home,Cumulative inflation rate,COVID death rate,New cases/1000 people,Active cases/1000 people,#days: decreasing COVID cases,% hospital bed utilization,Testing capacity,Tests done/1000 people,% ICU utilization,Ventilator shortage,Imported COVID cases
7420,2020-01-22,California,6071,0,0,0,0,0,6071,San Bernardino County,6,15,18,3.53,25.3,1.9,46.3,0.62,2.92,0,2171603,0.6,2020-01-22,1.5,16,60164,7.9,52.8,49.75,106,35,98,2.05,0.22,NaN,NaN,4,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0
7421,2020-01-23,California,6071,0,0,0,0,0,6071,San Bernardino County,6,13,17,3.62,25.6,2.0,48.5,0.63,2.99,0,2171603,3.0,2020-01-23,1.5,16,60164,7.9,52.8,49.75,106,35,98,2.05,0.22,NaN,NaN,4,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0
7422,2020-01-24,California,6071,0,0,0,0,0,6071,San Bernardino County,6,13,16,3.77,25.8,2.4,50.4,0.61,3.16,0,2171603,8.9,2020-01-24,1.5,16,60164,7.9,52.8,49.75,106,35,98,2.05,0.22,NaN,NaN,4,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0


In [101]:
new_cases_1k_la_list = CA_combined_sd_inertia_data_sb['New cases/1000 people'].values

### Finding mean, std. dev

In [102]:
meanVal= np.mean(new_cases_1k_la_list, axis=0)
stdVal = np.std(new_cases_1k_la_list, axis=0)
print('Mean: ', np.mean(new_cases_1k_la_list, axis=0))
print('Standard Deviation:', np.std(new_cases_1k_la_list, axis=0))

Mean:  0.09443490566037738
Standard Deviation: 0.12868128761243755


### Method to compute normal distribution to label the data 

In [103]:
  def determine_growth_label_stdev(i,meanVal,stdVal):
    if (i > (meanVal + (stdVal * 1) / 3)):
        #print("Label: ", i, "drastic_growth")
        return 'drastic_growth'
    elif ( i > (meanVal) and i <= (meanVal + (stdVal * 1) / 3)):
        #print("Label: ", i, "mild_growth")
        return 'mild_growth'
    elif (i >= (meanVal - (stdVal * 1) / 3) and i <= meanVal):
        #print("Label: ", i, "flat_growth")
        return 'flat_growth'
    elif (i < (meanVal - (stdVal * 1) / 3) and i >= (meanVal - (stdVal * 2) / 3)):
        #print("Label: ", i, "mild_decrement")
        return 'mild_decrement'
    elif (i < (meanVal - (stdVal * 2) / 3)):
        #print("Label: ", i, "drastic_decrement")
        return 'drastic_decrement'
    

In [104]:
CA_combined_sd_inertia_data_sb['growth_label'] = CA_combined_sd_inertia_data_sb['New cases/1000 people'].apply(lambda x: determine_growth_label_stdev(x,meanVal,stdVal) )
CA_combined_sd_inertia_data_sb.head(4)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,CTFIPS,CTNAME,STFIPS,Social distancing index,% staying home,Trips/person,% out-of-county trips,% out-of-state trips,Miles/person,Work trips/person,Non-work trips/person,New COVID cases,Population,% change in consumption,date,Transit mode share,% people older than 60,Median income,% African Americans,% Hispanic Americans,% Male,Population density,Employment density,# hot spots/1000 people,Hospital beds/1000 people,ICUs/1000 people,# contact tracing workers/1000 people,# contact tracing workers/1000 people rank,% people older than 60 rank,COVID exposure/1000 people,#days: decreasing ILI cases,Unemployment claims/1000 people,Unemployment rate,% working from home,Cumulative inflation rate,COVID death rate,New cases/1000 people,Active cases/1000 people,#days: decreasing COVID cases,% hospital bed utilization,Testing capacity,Tests done/1000 people,% ICU utilization,Ventilator shortage,Imported COVID cases,growth_label
7420,2020-01-22,California,6071,0,0,0,0,0,6071,San Bernardino County,6,15,18,3.53,25.3,1.9,46.3,0.62,2.92,0,2171603,0.6,2020-01-22,1.5,16,60164,7.9,52.8,49.75,106,35,98,2.05,0.22,NaN,NaN,4,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,drastic_decrement
7421,2020-01-23,California,6071,0,0,0,0,0,6071,San Bernardino County,6,13,17,3.62,25.6,2.0,48.5,0.63,2.99,0,2171603,3.0,2020-01-23,1.5,16,60164,7.9,52.8,49.75,106,35,98,2.05,0.22,NaN,NaN,4,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,drastic_decrement
7422,2020-01-24,California,6071,0,0,0,0,0,6071,San Bernardino County,6,13,16,3.77,25.8,2.4,50.4,0.61,3.16,0,2171603,8.9,2020-01-24,1.5,16,60164,7.9,52.8,49.75,106,35,98,2.05,0.22,NaN,NaN,4,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,drastic_decrement
7423,2020-01-25,California,6071,0,0,0,0,0,6071,San Bernardino County,6,27,21,3.29,27.1,2.3,45.3,0.31,2.99,0,2171603,3.0,2020-01-25,1.5,16,60164,7.9,52.8,49.75,106,35,98,2.05,0.22,NaN,NaN,4,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,1,54.0,0.0,0.0,0.0,0,0,drastic_decrement


### Export CSV File

In [105]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/CA/'
filename = 'CA_sd_inertia_labeledData_SanBernardino_'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
CA_combined_sd_inertia_data_sb.to_csv(location+filename,index=False)

CA_sd_inertia_labeledData_SanBernardino_Oct11.csv


In [106]:
print(CA_combined_sd_inertia_data_sb[CA_combined_sd_inertia_data_sb['growth_label'] == 'drastic_growth'].shape)
print(CA_combined_sd_inertia_data_sb[CA_combined_sd_inertia_data_sb['growth_label'] == 'mild_decrement'].shape)
print(CA_combined_sd_inertia_data_sb[CA_combined_sd_inertia_data_sb['growth_label'] == 'drastic_decrement'].shape)
print(CA_combined_sd_inertia_data_sb[CA_combined_sd_inertia_data_sb['growth_label'] == 'mild_growth'].shape)
print(CA_combined_sd_inertia_data_sb[CA_combined_sd_inertia_data_sb['growth_label'] == 'flat_growth'].shape)


(57, 54)
(53, 54)
(70, 54)
(10, 54)
(22, 54)
